<a href="https://www.kaggle.com/code/patrickleal/transformation-cleaning?scriptVersionId=145147145" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# **Importing Libraries**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re

# Import matplotlib for data visualization
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import warnings
warnings.filterwarnings('ignore')

In [ ]:
stadium_details_raw = pd.read_csv('/kaggle/input/brazilian-stadiums-dataset/brazilian-stadiums-details.csv')
stadiums_raw = pd.read_csv('/kaggle/input/brazilian-stadiums-dataset/brazilian-stadiums.csv')

In [ ]:
stadium_details_raw.head()

In [ ]:
stadiums_raw.head()

# **Functions Used**

**Function to transform the Date column:**

In [ ]:
def month_name_to_number(dataframe, col_index):
    ''' This function transforms the complete date in Portuguese 
    and returns the equivalent in dd-mm-yyyy '''
    
    # col_index is the index of the columns i want
    df = pd.DataFrame(dataframe.iloc[:, col_index]) # 1- creates a local dataframe with the column we want
    
    # creates a dict
    mon_to_number = {
    'janeiro': '01',
    'fevereiro': '02',
    'março': '03',
    'abril': '04',
    'maio': '05',
    'junho': '06',
    'julho': '07',
    'agosto': '08',
    'setembro': '09',
    'outubro': '10',
    'novembro': '11',
    'dezembro': '12'
    }
    
    column_name = df.columns[0] # 2- copy the label of the column
    
    for mon, num in mon_to_number.items():
        df[column_name] = df[column_name].str.replace(mon, num)                 # 3- replace the month name with the corresponding number
        df[column_name] = df[column_name].str.replace(str.capitalize(mon), num) # 4- replace the capitalized month name with the corresponding number   
        df[column_name] = df[column_name].str.replace(' de ', '-').str.strip()  # 5- replace ' de ' for '-'
    
    dataframe[column_name] = df[column_name]                                    # 6- adiciona a coluna limpa no dataframe
    return dataframe

# **Data Transformation**

## **Stadiums Dataset**

In [ ]:
stadiums_raw.info()

### **What to do?**

1. Change the labels of the columns.
1. Change the data type of the 'capacidade' column.
1. Replace 'Gov.' with 'Governo'

**1) Changing the labels of the columns**

In [ ]:
stadiums_df = stadiums_raw.copy() # create a copy os the raw dataset

# renaming de columns labels
stadiums_df.rename(columns={'Estádio': 'Stadium_Name', 
                           'Localidade': 'Locality',
                           'Unidade federativa': 'Federative_Units',
                           'Proprietário': 'Owner',
                           'Capacidade': 'Capacity'}, inplace=True) 
stadiums_df.head()

**2) Changing the data type of the 'capacidade':**

In [ ]:
stadiums_df['Capacity'] = stadiums_df['Capacity'].str.replace(' ','') # remove empty space 
stadiums_df['Capacity'] = stadiums_df['Capacity'].str.replace('.','') # remove the '.'
stadiums_df['Capacity'] = stadiums_df['Capacity'].astype(int)         # change to int 

In [ ]:
stadiums_df.info()

**3) Replace 'Gov.' with 'Governo'**

In [ ]:
stadiums_df['Owner'] = stadiums_df['Owner'].str.replace('Gov.', 'Governo').str.strip()
stadiums_df.head()

In [ ]:
stadiums_df.info()

## **Stadium Details Dataset**

In [ ]:
stadium_details_raw.info()

### **What to do?**

1. Change the column labels.
1. Transform the Date columns.
1. Transform the 'Publico Recorde' column.
1. Clean 'Stadium_Name'
1. Clean 'Official_Name'
1. Clean 'Nicknames'


In [ ]:
details_stadium_df = stadium_details_raw.copy() # creating a copy of the raw dataset
details_stadium_df.head()

**1) Renaming the Columns Labels**

In [ ]:
# choosing the new labels
new_labels = {'Estadio': 'Stadium_Name',
              'link': 'Wiki_Page_Link',
              'Nome Oficial': 'Official_Name',
              'Apelido': 'Nicknames',
              'Data Inauguracao': 'Opening_Date',
              'Público recorde': 'Record_Attendance',
              'Data recorde': 'Record_Date',
              'Partida com mais público': 'Match_with_the_highest_attendance'}

# renaming the columns
details_stadium_df.rename(columns=new_labels, inplace=True) 
details_stadium_df.head()

**2) Transforming the columns 'Date'**

In [ ]:
details_stadium_df[['Opening_Date', 'Record_Date']].head(15)

In [ ]:
#creating a df with the column 'Opening_Date'
# 1- removing all '()' and everything inside
Op_date_df = pd.DataFrame(details_stadium_df['Opening_Date'].str.replace(r'\([^)]*\)', '', regex=True))   

# 2- removing all '[]' and everything inside
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'\[[^\]]*\]', '', regex=True)  

# 3- removing all '\n' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'\n.*', '', regex=True)              

# 4- removing all 'e reinauguração' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'e reinauguração.*', '', regex=True) 

# 5- removing all 'Reinauguração' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'Reinauguração.*', '', regex=True)

# 6- removing all 'Original:'
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace('Original:', '')   

# 7- removing all 'Inauguração 1º' and replacing for '01'
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace('Inauguração 1º', '01')   

# 8- removing all '— ' and everything that comes after
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace(r'— .*', '', regex=True)     

# 9- removing all '1º' and replacing for '01'
Op_date_df['Opening_Date'] = Op_date_df['Opening_Date'].str.replace('1º', '01')                   


I know there's probably a better way to do this, but I couldn't figure it out.

I had to use chatGPT  for some Regex.

Now I need to obtain only the first occurrence where the complete date appears.

I know by reading the wiki page that this is the original opening date.

In [ ]:
concatenated_lines = []                        # 1- the list that will store the cleaned lines

for i in range(len(Op_date_df)):
    try:

        line = []                              # 2- the list  of the current line in the loop
        
                                               # 3- in this block i get the splited elements of the current line
        for ele in Op_date_df.iloc[i]:
            value = ele.split()
            line.append(value)
            break

        line = line[0][:5]                     # 4- save only the first 5 elements 
        concat_line = ' '.join(line)           # 5- Receives the line with the first 5 elements of each line concatenated
        concatenated_lines.append(concat_line) # 6- append the current line in the lists with the lines concatenated
    except Exception as error:
        concatenated_lines.append(np.nan)      # 7- append a nan if an error occurs
    
clean_Date = pd.Series(concatenated_lines)     # 8- create a Series with all cleaned lines

In [ ]:
Op_date_df['clean_OP_Date'] = clean_Date
Op_date_df['clean_OP_Date'].head(10)

Checking inconsistencies

In [ ]:
Op_date_df[['Opening_Date', 'clean_OP_Date']].loc[124:133]

Fixing two more lines

In [ ]:
# replacing with the correct date
Op_date_df['clean_OP_Date'] = Op_date_df['clean_OP_Date'].str.replace('198427', '1984')
Op_date_df['clean_OP_Date'] = Op_date_df['clean_OP_Date'].str.replace('19403', '1940')
Op_date_df.loc[124:133]

Now it is correct, time to add the new clean opening column to the main dataframe

In [ ]:
details_stadium_df['opening_date_cleaned'] = Op_date_df['clean_OP_Date']


In [ ]:
details_stadium_df[['Opening_Date', 'opening_date_cleaned']].head()

In [ ]:
details_stadium_df.iloc[:, 8].head()

In [ ]:
# Using the function to tranform the dates
month_name_to_number(details_stadium_df, 8)

In [ ]:
details_stadium_df[['Opening_Date','Record_Date', 'opening_date_cleaned']].head()

**Time to do the same with 'Record_Date' column**

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# creating a df with the 'Record_Date' column
rec_date_df = pd.DataFrame(details_stadium_df['Record_Date'])
rec_date_df['clean_rec_date'] = rec_date_df['Record_Date'].str.replace(r'\([^)]*\)', '', regex=True)   # 1- removing all '()' and everything inside

# Looking at the stadium's Wiki page, both dates had the same record attendance,
# so I decided to save the more recent one, which is November 2, 2010
rec_date_df['clean_rec_date'] = rec_date_df['clean_rec_date'].str.replace('29 de maio[3] e ', '')

rec_date_df['clean_rec_date'] = rec_date_df['clean_rec_date'].str.replace(r'\[[^\]]*\]', '', regex=True) # 2- removing all '[]' and everything inside
rec_date_df['clean_rec_date'] = rec_date_df['clean_rec_date'].str.replace('1º', '01')                    # 3- removing all '1º' and replacing for '01'

In [ ]:
details_stadium_df['record_date_cleaned'] = rec_date_df['clean_rec_date']
details_stadium_df[['Record_Date', 'record_date_cleaned']]

In [ ]:
# using the function to clean the date
month_name_to_number(details_stadium_df, 9)

details_stadium_df['record_date_cleaned'] = details_stadium_df['record_date_cleaned'].str.replace(' ', '-').str.replace('/', '-')
details_stadium_df[['Record_Date','record_date_cleaned']].head(20)

**3) Transforming the 'Record_Attendance' column**

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
# creating a df with the copy of the 'Record_Attendance' column
rec_attendance_df = pd.DataFrame(details_stadium_df['Record_Attendance'])

# removing some words
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['Record_Attendance'].str.replace('presentes', '')\
                                                                                  .str.replace('pagantes', '_').str.strip() # replaces a word for '_'

# removing the words 'pessoas'
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace('pessoas2', '')\
                                                                                     .str.replace('Pessoas', '')\
                                                                                     .str.replace('pessoas', ' ')

#removing some more words
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace('torcedores', '')\
                                                                                     .str.replace('espectadores', '')\
                                                                                     .str.replace('lugares', '')
# removing [] and everything inside
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace(r'\[[^\]]*\]', '', regex=True)

# removing more words
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace('(não oficial)', '')\
                                                                                     .str.replace('(de acordo com a CBF)', '')\
                                                                                     .str.replace('(público total)', '')

rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace('.', '') # remove dots
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace('  ', '+') # replace double spaces for '+'

# creates a dictionary with words and symbols i will remove or replace for oanother symbols
replace = {'pags': '', 'mil': '000', 'Mil':'', '\n': '-', 'e': '', ',': '', '\xa0': ''}
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].replace(replace, regex=True) # uses the dict created

# replaces the () for '-'
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace(r'[()]', '-', regex=True).str.strip()

# replace some more symbols
rec_attendance_df['clean_rec_attendance'] = rec_attendance_df['clean_rec_attendance'].str.replace(' ', '')\
                                                                                    .str.replace('+-', '-')\
                                                                                    .str.replace('_', '-')\
                                                                                    .str.replace('--', '-')\
                                                                                    .str.replace('+', '-')\
                                                                                    .str.replace('-', ' ')\
                                                                                    .str.strip()

# gets only the max number of each line 
rec_attendance_df['clean_rec_attendance'] = [max(linha.split()) if isinstance(linha, str) else np.nan
                                             for linha in rec_attendance_df['clean_rec_attendance']]
 

In [ ]:
# add the cleaned column to the main df
details_stadium_df['record_attendance_cleaned'] = rec_attendance_df['clean_rec_attendance']
details_stadium_df[['Record_Attendance', 'record_attendance_cleaned']].head(10)

**4) Cleaning 'Stadium_Name' column**

In [ ]:
# creates a new column in the main df and remove the word 'Sisbrace' and everything that comes after
details_stadium_df['stadium_name_cleaned'] = details_stadium_df['Stadium_Name'].str.replace(r'Sisbrace.*', '', regex=True)

# remove the [] and everything inside
details_stadium_df['stadium_name_cleaned'] = details_stadium_df['stadium_name_cleaned'].str.replace(r'\[[^\]]*\]', '', regex=True)

# remove the () in this particular line
details_stadium_df['stadium_name_cleaned'][27] = details_stadium_df['stadium_name_cleaned'][27].replace('(Manduzão)', 'Manduzão')

details_stadium_df[['Stadium_Name', 'stadium_name_cleaned']].head(15)

**5) Cleaning 'Official_Name' column**

In [ ]:
# creates a new column in the main df and remove the word 'Sisbrace' and everything that comes after
details_stadium_df['official_name_cleaned'] = details_stadium_df['Official_Name'].str.replace(r'Sisbrace.*', '', regex=True)

# remove the [] and everything inside
details_stadium_df['official_name_cleaned'] = details_stadium_df['official_name_cleaned'].str.replace(r'\[[^\]]*\]', '', regex=True).str.strip()

details_stadium_df[['Official_Name', 'official_name_cleaned']].head(10)

**6) Cleaning 'Nicknames' column**

In [ ]:
# creates a new column in the main df and remove the word 'Sisbrace' and everything that comes after
details_stadium_df['nicknames_cleaned'] = details_stadium_df['Nicknames'].str.replace(r'Sisbrace.*', '', regex=True)

# clean this particular line
details_stadium_df['nicknames_cleaned'][18] = details_stadium_df['nicknames_cleaned'][18].replace('Ilha', ' Ilha, ')\
                                                                                         .replace(',  ', ' ').strip()

# remove the [] and everything inside
details_stadium_df['nicknames_cleaned'] = details_stadium_df['nicknames_cleaned'].str.replace(r'\[[^\]]*\]', ' ', regex=True).str.strip()

# replace the '\n' for an empty space
details_stadium_df['nicknames_cleaned'] = details_stadium_df['nicknames_cleaned'].str.replace('\n', ' ')

# add an empty space before all capitalized word
details_stadium_df['nicknames_cleaned'] = details_stadium_df['nicknames_cleaned'].str.replace(r'([A-Z])([a-z])', r' \1\2', regex=True).str.strip()

# remove some symbols
details_stadium_df['nicknames_cleaned'] = details_stadium_df['nicknames_cleaned'].str.replace('"', '').str.replace('.', '').str.strip()
details_stadium_df['nicknames_cleaned'] = details_stadium_df['nicknames_cleaned'].str.replace('-', ' ').str.replace('/', ' ').str.replace('  ', ' ')

# clean this particular line
details_stadium_df['nicknames_cleaned'][40] = details_stadium_df['nicknames_cleaned'][40].replace('BelmiroA','Belmiro A')


In [ ]:
details_stadium_df[['Nicknames', 'nicknames_cleaned']].head(20)

Some stadiums have more than one nickname, they should be separated by commas but they are not, so I think this column won't be very useful for further analysis.

In [ ]:
wanted_columns = ['stadium_name_cleaned', 'official_name_cleaned', 'nicknames_cleaned', 'opening_date_cleaned',
                 'record_attendance_cleaned', 'record_date_cleaned', 'Match_with_the_highest_attendance']

#create a new df with only the transformed columns
stadium_details_cleaned = details_stadium_df[wanted_columns]

The new df:

In [ ]:
stadium_details_cleaned

# **Data Cleaning**

## **Stadiums Dataset**

In [ ]:
stadiums_df.info()

Looks like there is no missing values in this dataset and the types are correct

In [ ]:
stadiums_df[stadiums_df.duplicated() == True]

There is no Duplicated 

## **Stadium Details Dataset**

In [ ]:
stadium_details_cleaned.info()

**What to do?**

1. Remove the missing dates
1. Convert date columns to the 'date' type
1. Convert the 'record_attendance_cleaned' column to int

**1) Remove the missing dates**

In [ ]:
stadium_details_cleaned[stadium_details_cleaned['opening_date_cleaned'].isna()]

These rows have a lot more missing data.

In [ ]:
stadium_details_cleaned = stadium_details_cleaned.dropna(subset=['opening_date_cleaned', 'record_date_cleaned'],
                                                         ignore_index=True)

**2) Convert date columns to the 'date' type**

In [ ]:
pd.set_option('display.max_rows', None)

**Opening Date column**

In [ ]:
opening_date_cleaned = stadium_details_cleaned['opening_date_cleaned']
indices = [indice for indice, elemento in enumerate(stadium_details_cleaned['opening_date_cleaned'])
           if len(elemento) <= 4]

In [ ]:
stadium_details_cleaned['opening_date_cleaned'][indices]

Four stadiums have only the year for the opening date, i will keep them and create a dummy month and day.

In [ ]:
stadium_details_cleaned['opening_date_cleaned'] = pd.to_datetime(stadium_details_cleaned['opening_date_cleaned'], format='mixed')

**Record Date column**

Only a few dates do not have the complete date, i will keep them too.

In [ ]:
stadium_details_cleaned['record_date_cleaned'] = pd.to_datetime(stadium_details_cleaned['record_date_cleaned'], format='mixed')

**3) Convert the 'record_attendance_cleaned' column to int**

There is only one missing value in the record_attendance_cleaned column, so i will drop it

In [ ]:
stadium_details_cleaned = stadium_details_cleaned.dropna(subset='record_attendance_cleaned')
stadium_details_cleaned['record_attendance_cleaned'] = pd.to_numeric(stadium_details_cleaned['record_attendance_cleaned'], errors='coerce')

In [ ]:
stadium_details_cleaned.info()

Now the types are correct

In [ ]:
stadiums_df.head()

In [ ]:
#removing the '_cleaned' in the columns names
stadium_details_cleaned.columns = stadium_details_cleaned.columns.str.replace('_cleaned', '')
stadium_details_cleaned.columns

Saving the cleaned dfs to csv

In [ ]:
stadiums_df.to_csv('/kaggle/working/stadiums.csv', index=False)
stadium_details_cleaned.to_csv('/kaggle/working/stadium_details.csv', index=False)